In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./menu.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                resp = requests.get(input_.loc[a, 'Url'],
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                html = etree.HTML(str(soup))
                
                # = = = = = = = = = = = = = = =
                
                type_ = html.xpath('//span[@id="lblPartTypeName"]/text()')[0].strip()
                
                # = = = = = = = = = = = = = = =
                
                part_number = html.xpath('//span[@id="lblProductName"]/text()')[0].strip()
                
                # = = = = = = = = = = = = = = =
                
                description = html.xpath('//span[@id="lblProductDesc"]/text()')[0].strip()
                
                # = = = = = = = = = = = = = = =
                                
                application_summary = ''.join(html.xpath('//div[@id="divAppSummary"]/p/text()')).strip().replace(', ', '\n')

                # = = = = = = = = = = = = = = =
                
                src = html.xpath('//a[@id="ProductPic"]/@href')[0].strip()
                
                # = = = = = = = = = = = = = = =
                
                vehicle_url = html.xpath('//input[@id="hidfDetailApp"]/@value')
                if len(vehicle_url) == 0:
                    vehicle_url = ''
                else:
                    vehicle_url = 'https://www.dormanproducts.com/' + vehicle_url[0].strip()
                
                # = = = = = = = = = = = = = = =
                
                list_oe = [oe.strip() for oe in html.xpath('//section[@id="productOE"]/descendant::tbody/tr/th/text()')]
                oe = ';'.join(sorted(list(set(list_oe))))
                
                # = = = = = = = = = = = = = = =
                
                list_tr = html.xpath('//section[@id="productSpec"]/descendant::tr')

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'序号': int(input_.loc[a, 'No.']),
                                         'Part_Number': part_number,
                                         'Type': type_,
                                         'Description': description,
                                         'Application_Summary': application_summary,
                                         'Vehicle_1': '',
                                         'Vehicle_2': '',
                                         'OE': oe,
                                         'Pic': '',
                                         'Url': input_.loc[a, 'Url'],
                                         'Vehicle_Url': vehicle_url,
                                         'Src': src}])

                for tr in list_tr:
                    name = tr.xpath('./th/text()')[0].lower().strip()[:-1]
                    df_temp.loc[0, name[:-1].strip() if name.endswith(':') else name] = tr.xpath('./td/text()')[0].strip()

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'No.': int(input_.loc[a, 'No.']),
                                     'Url': input_.loc[a, 'Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Url'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(10):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['序号'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./part-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['No.'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./part_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：1

[ok] - https://www.dormanproducts.com/p-127655-976-720.aspx?origin=keyword
[尝试次数：3] - [剩余数量：0] - [当前时间：11:26:28]

输出ing...

Done ~
